In [150]:
# Identify athletes meeting OCTC selection rules

#1. At least 1 athlete per event per gender
#2. Except for 100m, 400m, cap at 3 athletes per event. 100m, 400m capped at 6
#3. Where top athlete is >30 yrs old (except marathon), to include next athlete as well (below 30)
#4. Where althlete qualified in 2 events, to choose the better performing one
#5. For athletes looking to do full time, to write in to SAA for special consideration
#6. Exclude SPEX carded athletes
#7. Except for marathon, age threshold cut off of 40 yrs old for top athlete
#8. No double tapping of prog - potential names in red


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic

from google.cloud import storage



In [151]:
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION
FROM `saa-analytics.results.athlete_results_prod` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [152]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,23/4/97,,,None,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,1/8/19,nan,0.5,None,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,3/3/74,nan,,Throw,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,28/12/02,IND,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,9/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,6/9/10,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,20/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local


In [153]:
# Create mm/dd/yy column for date of event

import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(competitors)):
        
    rowIndex = competitors.index[i]

    date = competitors.iloc[rowIndex,16]
    year = competitors.iloc[rowIndex,17]    
    
    if 'to' in date or ' - ' in date:
        
        pos = re.search('to|\s\-\s', date)
                                                
        # Splice string to day and month

        split_pos_start=pos.start()+3

            
        final_date = date[split_pos_start:] # left string post splicing
        final_year = year[2:]
        
        event_date = final_date + '/' + final_year
        
        competitors.loc[rowIndex, 'event_date'] = event_date
        
    elif re.search('\w\-\w', date):
        
        if competitors.iloc[rowIndex, 15] == "National School Games":
        
            event_date = '04'+'/'+date[1:3] + '/' + year[2:]  # reverse order from dd/mm to mm/dd
        
            competitors.loc[rowIndex, 'event_date'] = event_date
        
        else:
            
            event_date = date + '-' + year[2:]
            
            competitors.loc[rowIndex, 'event_date'] = event_date

        
        

        
competitors['event_date'] = competitors['event_date'].astype(str)
competitors['event_date'] = competitors['event_date'].str.replace('\xa0', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\r', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\n', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.strip()
       
    

In [154]:
# Calculate number of days from today to event date

competitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], errors='coerce')

competitors['delta_time']= datetime.datetime.now() - competitors['event_date_dt']

competitors['delta_time_conv'] = pd.to_numeric(competitors['delta_time'].dt.days, downcast='integer')


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_58312/2667859050.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  competitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], errors='coerce')


In [155]:
# Choose last 6 months or 12 months of results 

competitors = competitors[(competitors['delta_time_conv']>=0) & (competitors['delta_time_conv']<=365)]

competitors=competitors.reset_index(drop=True)


In [156]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Athletics/')


competitors.to_csv('athletes_time_filtered.csv', sep=',', encoding='utf-8-sig', index=False)

In [157]:
competitors[competitors['NAME']=='SOH RUI YONG GUILLAUME']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv
611,SOH RUI YONG GUILLAUME,04:59.3,INDIVIDUAL,33,12,None,1 mile,None,None,None,...,Mid,Male,Vitaly West Mininster Mile 5K (UK),22-Sep,2024,International,22-Sep-24,2024-09-22,215 days 09:45:06.849222,215.0
612,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,None,...,Long,Male,"Tokai University Long Distance Challenge 10,0...",25-Nov,2024,International,25-Nov-24,2024-11-25,151 days 09:45:06.849222,151.0
613,SOH RUI YONG GUILLAUME,15:07.7,INDIVIDUAL,33,46,None,5000m,None,None,None,...,Long,Male,Friday Night Under the Lights 5K (UK),13-Sep,2024,International,13-Sep-24,2024-09-13,224 days 09:45:06.849222,224.0
614,SOH RUI YONG GUILLAUME,08:35.8,INDIVIDUAL,33,None,None,3000m,None,None,None,...,Long,Male,Hercules Wimbledon Athletics Club 3000m,14-Jun,2024,International,14-Jun-24,2024-06-14,315 days 09:45:06.849222,315.0


In [158]:
mask = (competitors['DIVISION'].str.contains(r'B', na=False) & competitors['COMPETITION'].str.contains(r'National School Games', na=False))
competitors.loc[mask, 'AGE'] = '16'

In [159]:
competitors['DOB_new'] = pd.to_datetime(competitors['DOB'], errors='coerce')

competitors['year_extract']=competitors['DOB_new'].dt.strftime('%Y')

competitors['year_extract']=pd.to_numeric(competitors['year_extract'])

competitors['age_extract'] = 2025 - competitors['year_extract']


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_58312/3522909459.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  competitors['DOB_new'] = pd.to_datetime(competitors['DOB'], errors='coerce')


In [160]:
def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


competitors['age_extract'] = competitors['age_extract'].apply(age)


In [161]:
# If NSG event then choose AGE otherwise choose age_extract

condition1=competitors['COMPETITION']=='National School Games'


competitors['age_extract']=competitors['AGE'].where((condition1), competitors['age_extract'].values)


In [162]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,DOB_new,year_extract,age_extract
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,11/9 to 11/10,2024,Local,11/10/24,2024-11-10,166 days 09:45:06.849222,166.0,1990-11-24,1990.0,35.0
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,11/9 to 11/10,2024,Local,11/10/24,2024-11-10,166 days 09:45:06.849222,166.0,1995-04-05,1995.0,30.0
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,9/7 to 9/7,2024,Local,9/7/24,2024-09-07,230 days 09:45:06.849222,230.0,1997-04-23,1997.0,28.0
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,6/22 to 6/22,2024,Local,6/22/24,2024-06-22,307 days 09:45:06.849222,307.0,2019-01-08,2019.0,6.0
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,6/22 to 6/22,2024,Local,6/22/24,2024-06-22,307 days 09:45:06.849222,307.0,2074-03-03,2074.0,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13873,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,2/23 to 3/2,2025,Local,3/2/25,2025-03-02,54 days 09:45:06.849222,54.0,2002-12-28,2002.0,23.0
13874,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,2/23 to 3/2,2025,Local,3/2/25,2025-03-02,54 days 09:45:06.849222,54.0,2009-09-07,2009.0,16.0
13875,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,2/23 to 3/2,2025,Local,3/2/25,2025-03-02,54 days 09:45:06.849222,54.0,2010-06-09,2010.0,15.0
13876,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,2/23 to 3/2,2025,Local,3/2/25,2025-03-02,54 days 09:45:06.849222,54.0,2009-07-20,2009.0,16.0


In [163]:
competitors['AGE'] = competitors['AGE'].replace(regex=r'-', value=np.NaN)
competitors['AGE'] = competitors['AGE'].replace(regex=r'nan', value=np.NaN)


competitors['AGE'] = pd.to_numeric(competitors['AGE'])

#athletes_selected = competitors[((competitors['AGE']>=15) & (competitors['AGE']<=17))]

#athletes_selected = competitors[(competitors['YEAR']=='2024')]

In [164]:
# Converts any time format into seconds and fixed inconsistent time formats

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
   # print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
        #    print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
                
                else:
    
                    output=float(str(metric))
        
            elif string=='':   # no event description at all!
                
                output='' # return nothing
            
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                # fix different formats
            
                if count==0:
                
                    output=float(substring)
            
            
                elif '10,000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    

                elif '1500m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                    
                    if len(substring)==7:  # format is X:XX:XX and not XX:XX:XX 
                        
                        idx = 4 # 5th character position
                        replacement = "."
                        metric = '0' + metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    
                        
                    else:  # format is XX:XX:XX
                        
                        idx = 5 # 5th character position
                        replacement = "."
                        metric = metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    
                    
        
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output

In [165]:
athletes=competitors

In [166]:
'''

for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()

# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'50', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'60', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'80', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'

mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'

mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'

mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'^600$', na=True, regex=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'^3000m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'^5000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

#mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles



mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles|Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.838|0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles(0.838m)'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['REGION'].str.contains('International', na=False) & athletes['EVENT_CLASS'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles(0.838m)'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles(0.838m)'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles(0.838m)'



mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('1.067', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'
mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains('International', na=False) & athletes['EVENT_CLASS'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))  # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'



mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'
#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'



mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'


mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False)  & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'



mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False)  & athletes['REGION'].str.contains('International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'



# Throws


mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'600', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw(600g)'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) #NEW
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw(600g)'


mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'800', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw(800g)'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw(800g)'



mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(4kg)'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(7.26kg)'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['DIVISION'].str.contains(r'Open', na=False)))# NEW
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(4kg)'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Male') & (athletes['DIVISION'].str.contains(r'Open', na=False)))# NEW
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(7.26kg)'


mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw(4kg)'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False, regex=True) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw(7.26kg)'

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw|Discus', na=False) & athletes['EVENT_CLASS'].str.contains(r'2', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(2kg)'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw|Discus', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.00kg|1kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(1kg)'

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw|Discus', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # New
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(2kg)'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw|Discus', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # New
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(1kg)'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump|High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$|^Long\sjump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump|Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault|Pole vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'3000m steeplechase', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase(0.762m)'
mask = (athletes['EVENT'].str.contains(r'3000m steeplechase', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase(0.914m)'

mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False)  & athletes['DISTANCE'].str.contains(r'3000', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase(0.762m)'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False)  & athletes['DISTANCE'].str.contains(r'3000', na=False) & athletes['GENDER'].str.contains(r'Male', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase(0.914m)'


#mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

#mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'1500', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'


#mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
mask = (athletes['EVENT'].str.contains(r'5000m Race Walk', na=True) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'

mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'5000', na=False) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'

mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'10000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

#mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'

'''


"\n\nfor col in athletes.columns:\n    athletes[col] = athletes[col].astype(str)\n    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)\n    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)\n    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)\n    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)\n    athletes[col] = athletes[col].str.strip()\n\n# Create temporary mapped event column\n\nathletes['MAPPED_EVENT']=''\n\n# Correct javelin category\n\nmask = athletes['EVENT'].str.contains(r'Javelin', na=True)\nathletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'\n\n\n# Running\n\n#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'50', na=True))\n#athletes.loc[mask, 'MAPPED_EVENT'] = '50m'\n#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'60', na=True))\n#athletes.loc[mask, 'MAPPED_EVENT'] = '60m'\n#mask = (athletes['EVENT'].s

In [167]:
# NEW MAPPING
# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()


# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
#mask = athletes['EVENT'].str.contains(r'3000m', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'^5000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10\,000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

#mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles

#mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'


##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'



mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.838|0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = ((athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False)) 
         & ((athletes['EVENT_CLASS'].str.contains('None', na=False))|(athletes['EVENT_CLASS']==np.nan)|(athletes['EVENT_CLASS']=='')) 
         & athletes['REGION'].str.contains(r'International', na=False) & (athletes['DIVISION'].str.contains(r'None', na=False)))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

#mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


# Using np.where instead

#athletes['MAPPED_EVENT'] = np.where(((athletes['EVENT']=='110m hurdles|110m Hurdles') & ((athletes['EVENT_CLASS']=='')|athletes['EVENT_CLASS']=='None') & (athletes['REGION']=='International')), '110m Hurdles', ' ')   
                                


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'







#mask = (athletes['EVENT'].str.contains(r'^400m\sHurdles$', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = ' '
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84cm', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open|Invitational', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
##mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)& athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('None|0.762|0.914', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'



# Throws


#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'600g', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'800g', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS']=='4kg'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7.26', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'



#mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'7.26kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'4.00kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'



#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=False))  & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'


#mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=False)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'2kg|2.00kg', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1kg|1.00kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'None', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

#mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'1500', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'


#mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'10000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'



In [168]:
# Convert results and seed into seconds format

athletes.reset_index(drop=True, inplace=True)

for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()

for i in range(len(athletes)):
    
    result_out=''
    
        
    rowIndex = athletes.index[i]

    event=athletes.loc[rowIndex,'MAPPED_EVENT']    # event description
    
    result=athletes.loc[rowIndex,'RESULT'] # result
    
    if result=='—' or result=='DQ' or result=='SCR' or result=='FS' or result=='DNQ' or result=='DNS' or result=='NH' or result=='NM' or result=='FOUL' or result=='DNF' or result=='SR' :
        continue
    
    result_out = convert_time(i, event, result)
         
    athletes.loc[rowIndex, 'RESULT_CONV'] = result_out


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_58312/1719495126.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  athletes.loc[rowIndex, 'RESULT_CONV'] = result_out


In [169]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Athletics/')

athletes.to_csv("athletes.csv", encoding='utf-8')

In [170]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Athletics/')

benchmarks = pd.read_csv("26th Asian Athletics Benchmarks.csv")


In [171]:
benchmarks

,EVENT,GENDER,BENCHMARK
0,100m,Male,10.33
1,100m,Female,11.66
2,200m,Male,21.01
3,200m,Female,23.66
4,400m,Male,46.29
5,400m,Female,54.4
6,800m,Male,01:51.5
7,800m,Female,02:08.4
8,1500m,Male,03:47.4
9,1500m,Female,04:28.8


In [172]:
for col in benchmarks.columns:
    benchmarks[col] = benchmarks[col].astype(str)
    benchmarks[col] = benchmarks[col].str.replace('\xa0', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('\r', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('\n', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.strip()


In [173]:
#SEAG_filtered=SEAG[SEAG['HEAT'].isnull() & SEAG['SUB_EVENT'].isnull()]

In [174]:
# Convert time for benchmarks and results

for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string = benchmarks.iloc[rowIndex,0]
    
    metric=benchmarks.iloc[rowIndex,2]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
    
    print(rowIndex, input_string, out)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

0 100m 10.33
1 100m 11.66
2 200m 21.01
3 200m 23.66
4 400m 46.29
5 400m 54.4
6 800m 111.5
7 800m 128.4
8 1500m 227.4
9 1500m 268.8
10 5000m 865.3
11 5000m 967.2
12 3000m Steeplechase 536.7
13 3000m Steeplechase 644.9
14 10,000m 1859.2
15 10,000m 2038.9
16 110m Hurdles 13.84
17 100m Hurdles 13.5
18 400m Hurdles 50.94
19 400m Hurdles 58.65
20 High Jump 2.2
21 High Jump 1.77
22 Long Jump 7.81
23 Long Jump 6.17
24 Triple Jump 16.07
25 Triple Jump 13.2
26 Shot Put 18.26
27 Shot Put 15.27
28 Discus Throw 56.61
29 Discus Throw 52.88
30 Javelin Throw 77.81
31 Javelin Throw 53.35
32 Hammer Throw 69.06
33 Hammer Throw 54.8
34 Pole Vault 5.34
35 Pole Vault 4.03
36 Decathlon 6308.0
37 Heptathlon 5381.0
38 4 x 100m 40.0
39 4 x 100m 45.36
40 4 x 400m 195.2
41 4 x 400m 224.3
42 4 x 400m 207.3


In [175]:
benchmarks

,EVENT,GENDER,BENCHMARK,Metric
0,100m,Male,10.33,10.33
1,100m,Female,11.66,11.66
2,200m,Male,21.01,21.01
3,200m,Female,23.66,23.66
4,400m,Male,46.29,46.29
5,400m,Female,54.4,54.40
6,800m,Male,01:51.5,111.50
7,800m,Female,02:08.4,128.40
8,1500m,Male,03:47.4,227.40
9,1500m,Female,04:28.8,268.80


In [176]:
mask = benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put|Jump|Throw|pole|Put|Decathlon|Heptathlon', na=True)

benchmarks.loc[mask, '2%']=benchmarks['Metric']*0.98
benchmarks.loc[~mask, '2%']=benchmarks['Metric']*1.02
benchmarks.loc[mask, '5%']=benchmarks['Metric']*0.95
benchmarks.loc[~mask, '5%']=benchmarks['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [177]:
benchmarks

,EVENT,GENDER,BENCHMARK,Metric,2%,5%
0,100m,Male,10.33,10.33,10.5366,10.8465
1,100m,Female,11.66,11.66,11.8932,12.2430
2,200m,Male,21.01,21.01,21.4302,22.0605
3,200m,Female,23.66,23.66,24.1332,24.8430
4,400m,Male,46.29,46.29,47.2158,48.6045
5,400m,Female,54.4,54.40,55.4880,57.1200
6,800m,Male,01:51.5,111.50,113.7300,117.0750
7,800m,Female,02:08.4,128.40,130.9680,134.8200
8,1500m,Male,03:47.4,227.40,231.9480,238.7700
9,1500m,Female,04:28.8,268.80,274.1760,282.2400


In [178]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes, 
    right=benchmarks,
    how='left',
    left_on=['MAPPED_EVENT', 'GENDER'],
    right_on=['EVENT', 'GENDER'],
)

In [179]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,DOB_new,year_extract,age_extract,MAPPED_EVENT,RESULT_CONV,EVENT_y,BENCHMARK,Metric,2%,5%
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34.0,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,1990-11-24,1990.0,35.0,Hammer Throw,32.4,Hammer Throw,69.06,69.06,67.6788,65.6070
1,"Koh, Wei Shien",4.41m,NUS,29.0,3,Open,Pole Vault,0,,L435B06,...,1995-04-05,1995.0,30.0,Pole Vault,4.41,Pole Vault,5.34,5.34,5.2332,5.0730
2,"Wen Qiang, Koh",07:38.0,SIT,27.0,4,Tertiary Men,Run,2400,,,...,1997-04-23,1997.0,28.0,400m,458.0,400m,46.29,46.29,47.2158,48.6045
3,"Harinee, Sivakumaru",15.03,SKM,4.0,7,5-Jun,Run,60,,nan,...,2019-01-08,2019.0,6.0,,,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50.0,5,Masters,Javelin Throw,0,,nan,...,2074-03-03,2074.0,51.0,,,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13873,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23.0,2,Open,Relay,400,Trial,None,...,2002-12-28,2002.0,23.0,4 x 100m,42.06,4 x 100m,40,40.00,40.8000,42.0000
13874,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16.0,2,Open,Relay,400,Trial,None,...,2009-09-07,2009.0,16.0,4 x 100m,42.06,4 x 100m,40,40.00,40.8000,42.0000
13875,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15.0,2,Open,Relay,400,Trial,None,...,2010-06-09,2010.0,15.0,4 x 100m,42.06,4 x 100m,40,40.00,40.8000,42.0000
13876,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16.0,2,Open,Relay,400,Trial,None,...,2009-07-20,2009.0,16.0,4 x 100m,42.06,4 x 100m,40,40.00,40.8000,42.0000


In [180]:
df[df['MAPPED_EVENT']=='10,000m']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,DOB_new,year_extract,age_extract,MAPPED_EVENT,RESULT_CONV,EVENT_y,BENCHMARK,Metric,2%,5%
495,DARREN CHONG SHUN HAO,37:54:00,SINGAPORE,31.0,25,None,10000m,10000m,None,None,...,1993-01-01,1993.0,32.0,"10,000m",2274.0,"10,000m",30:59.2,1859.2,1896.384,1952.160
496,LEE VANESSA YING ZHUANG,35:55:00,SINGAPORE,26.0,8,None,10000m,None,None,None,...,1998-01-01,1998.0,27.0,"10,000m",2155.0,"10,000m",33:58.9,2038.9,2079.678,2140.845
497,VANESSA LEE YING ZHUANG,40:21:00,SINGAPORE,26.0,10,None,10000m,None,None,None,...,1998-01-01,1998.0,27.0,"10,000m",2421.0,"10,000m",33:58.9,2038.9,2079.678,2140.845
612,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33.0,133,None,10000m,None,None,None,...,1991-01-01,1991.0,34.0,"10,000m",1871.4,"10,000m",30:59.2,1859.2,1896.384,1952.160
615,DANIEL LEOW SOON YE,35:10:00,INDIVIDUAL,27.0,7,None,10000m,None,None,None,...,1997-01-01,1997.0,28.0,"10,000m",2110.0,"10,000m",30:59.2,1859.2,1896.384,1952.160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10579,"Ahmed, Nawaz",39:49.8,Lacticbuds,22.0,11,Open,Run,10000,None,N034I03,...,2003-05-09,2003.0,22.0,"10,000m",2389.8,"10,000m",30:59.2,1859.2,1896.384,1952.160
10580,"Ko, Keane",33:51.9,Club ZOOM,25.0,1,Open,Run,10000,None,K326A00,...,2000-07-26,2000.0,25.0,"10,000m",2031.9,"10,000m",30:59.2,1859.2,1896.384,1952.160
10617,"NG, YEW CHEO",48:27.8,Cougars Athletic Association,39.0,2,Open,Run,10000,None,Y971Z86,...,1986-11-30,1986.0,39.0,"10,000m",2907.8,"10,000m",33:58.9,2038.9,2079.678,2140.845
10618,"Teoh, Katelyn",49:09.4,National University Singapore,20.0,3,Open,Run,10000,None,K831W05,...,2005-03-14,2005.0,20.0,"10,000m",2949.4,"10,000m",33:58.9,2038.9,2079.678,2140.845


In [181]:
# Change to numeric

df['RESULT_CONV'] = df['RESULT_CONV'].replace(regex=r'-', value=np.NaN)
df['RESULT_CONV'] = df['RESULT_CONV'].replace(regex=r'-', value=np.NaN)
df['RESULT_CONV'] = df['RESULT_CONV'].replace(regex=r'nan', value=np.NaN)

df['age_extract'] = df['age_extract'].replace(regex=r'-', value=np.NaN)
df['age_extract'] = df['age_extract'].replace(regex=r'-', value=np.NaN)
df['age_extract'] = df['age_extract'].replace(regex=r'nan', value=np.NaN)
df['age_extract'] = df['age_extract'].replace(regex=r'None', value=np.NaN)


df[['2%', '5%', 'RESULT_CONV']] = df[['2%', '5%', 'RESULT_CONV']].apply(pd.to_numeric)

df.to_csv("testing.csv", encoding='utf-8')


In [182]:
mask = df['CATEGORY_EVENT'].str.contains(r'Jump|Throw|jump|throw|Decathlon|Heptathlon|decathlon|heptathlon', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_CONV']-df['2%']
df.loc[mask, 'Delta5'] = df['RESULT_CONV']-df['5%']

df.loc[mask, 'Delta_Benchmark'] = df['RESULT_CONV']-df['Metric']

df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_CONV']
df.loc[~mask, 'Delta5'] =  df['5%'] - df['RESULT_CONV']

df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_CONV']

#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']


In [183]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta_Benchmark']/df['Metric']*100

In [184]:
#df['TIER'] = np.where((df['Delta_Benchmark']>=0), 'Tier 1',    
#            np.where(((df['Delta_Benchmark']<0) & (df['Delta2']>=0)), 'Tier2', ' '))

df['TIER'] = np.where((df['Delta_Benchmark']>=0), 'Tier 1',    
            np.where(((df['Delta_Benchmark']<0) & (df['Delta2']>=0)), 'Tier 2',
            np.where(((df['Delta2']<0) & (df['Delta5']>=0)), 'Tier 3', ' ')))


In [185]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Athletics/')

df.to_csv('asian_athletics_mapped.csv', sep=',', encoding='utf-8-sig', index=False)


In [186]:
#df[['age_extract']] = df[['age_extract']].apply(pd.to_numeric)

#athletes_selected = df[((df['age_extract']>=15) & (df['age_extract']<=17))]


In [187]:
athletes_selected=df

In [188]:
# Read name variations from GCS name lists bucket (Still in beta)


athletes_selected['NAME'] = athletes_selected['NAME'].str.replace('\xa0', '', regex=True)
athletes_selected['NAME'] = athletes_selected['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
athletes_selected['NAME'] = athletes_selected['NAME'].str.replace('\r', '', regex=True)
athletes_selected['NAME'] = athletes_selected['NAME'].str.replace('\n', '', regex=True)
athletes_selected['NAME'] = df['NAME'].str.strip()


# Read csv from GCS bucket

file_path = "gs://name_variations/name_variations.csv"
names = pd.read_csv(file_path,
                 sep=",",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})

# Iterate over dataframe and replace names

for index, row in names.iterrows():
        
    athletes_selected['NAME'] = athletes_selected['NAME'].replace(regex=rf"^{row['VARIATION']}$", value=f"{row['NAME']}")


In [189]:
# Choose the best result for each event participated by every athlete

best_selection = athletes_selected.sort_values(['MAPPED_EVENT', 'NAME','PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)

athletes=best_selection

In [190]:
# Exclude foreigners from MALAYSIA, THAILAND etc.

final_asian_selected = athletes[(athletes['TEAM']!='Malaysia')&(athletes['TEAM']!='THAILAND')&(athletes['TEAM']!='China') 
                       &(athletes['TEAM']!='South Korea')&(athletes['TEAM']!='Laos') 
                       &(athletes['TEAM']!='Philippines')&(athletes['TEAM']!='Piboonbumpen Thailand') 
                       &(athletes['TEAM']!='Chinese Taipei')&(athletes['TEAM']!='Gurkha Contingent') 
                       &(athletes['TEAM']!='Australia')&(athletes['TEAM']!='Piboonbumpen Thailand') 
                       &(athletes['TEAM']!='Hong Kong')&(athletes['TEAM']!='PERAK')&(athletes['TEAM']!='Sri Lanka') 
                       &(athletes['TEAM']!='Indonesia')&(athletes['TEAM']!='THAILAND')&(athletes['TEAM']!='MALAYSIA') 
                       &(athletes['TEAM']!='PHILIPPINES') & (athletes['TEAM']!='SOUTH KOREA')&(athletes['TEAM']!='Waseda') 
                       &(athletes['TEAM']!='LAOS')&(athletes['TEAM']!='CHINESE TAIPEI')
                       &(athletes['TEAM']!='INDIA')&(athletes['TEAM']!='Hong Kong, China')&(athletes['TEAM']!='AIC JAPAN')] 


In [191]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Athletics/')


final_asian_selected.to_csv('asian_selection_12M.csv', sep=',', encoding='utf-8-sig', index=False)


# Clean manual load data

In [4]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/SEA Games/')


df_SHA = pd.read_csv("SEA_manual_additions.csv", encoding='latin-1')

In [5]:
df_SHA

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,NaN,NaN,NaN,Agus Prayogo,1.0,NaN,NaN,NaN,2:32:59,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
1,NaN,NaN,NaN,ÊArlan Estobo Arbois,2.0,NaN,NaN,NaN,2:33:27,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
2,NaN,NaN,NaN,Nguyen Thanh Hoang,3.0,NaN,NaN,NaN,2:35:49,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
3,NaN,NaN,NaN,Tan Huong Leong,4.0,NaN,NaN,NaN,2:40:26,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
4,NaN,NaN,NaN,Vanh Pheara,5.0,NaN,NaN,NaN,2:41:26,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
5,NaN,NaN,NaN,Quoc Luong Trinh,6.0,NaN,NaN,NaN,2:41:36,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
6,NaN,NaN,NaN,Sanchai Namkhet,7.0,NaN,NaN,NaN,2:43:47,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
7,NaN,NaN,NaN,Joanito Fernandes,8.0,NaN,NaN,NaN,2:52:41,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
8,NaN,NaN,NaN,Tony Ah-Thit Payne,NaN,NaN,NaN,NaN,DNF,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
9,NaN,NaN,NaN,Yang Piseth,NaN,NaN,NaN,NaN,DNF,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN


In [6]:
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\xa0', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\r', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\n', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.strip()


In [7]:
df_SHA.to_csv('SEA_manual_additions.csv', index=False, encoding='utf-8')

# Read a csv of name list variations

In [2566]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

names = pd.read_csv("name_variations.csv")

In [2567]:
names

,VARIATION,NAME
0,^Harry\sIrfan\sCurra$,HARRY IRFAN CURRAN
1,"Curran, Harry Irfann",HARRY IRFAN CURRAN
2,"^Curran,\sHarry$",HARRY IRFAN CURRAN
3,Harry Irfan Curran Q,HARRY IRFAN CURRAN
4,Wei Jun Huang,HUANG WEI JUN
...,...,...
182,Chua Hsin-Wen Clara,"Chua Hsin-Wen, Clara"
183,Chua Hsin-Wen Clara Q,"Chua Hsin-Wen, Clara"
184,"HSIN-WEN CLARA, Chua","Chua Hsin-Wen, Clara"
185,CHUA HSIN-WEN CLARA,"Chua Hsin-Wen, Clara"


In [ ]:
lines = [line.strip() for line in open('file.csv')]
for x in lines:
    match=re.search(r'^_.*_$',x)
    if match: print x

In [ ]:
df['NAME'] = df['NAME'].replace(regex=r'^Harry\sIrfan\sCurra$', value='HARRY IRFAN CURRAN')


In [2568]:
for index, row in names.iterrows():
    print(row['VARIATION'])
    

^Harry\sIrfan\sCurra$
Curran, Harry Irfann
^Curran,\sHarry$
Harry Irfan Curran Q
Wei Jun Huang
HUANG, WEI JUN
., Hariharan
S/O KrishnS/O Krishna, Hariharan
KRISHNAN, HARIHARAN S/O
Andrew George Medina
Medina, Andrew
Medina, Andrew George
George Medina, Andrew
George George Medina, Andrew
Gabriel Lee
Lee, Gabriel
Jun Yu Low
Low Jun Yu
Low, Jun Yu
Caleb Hia
Melvin, Wong
MELVIN, WONG
Conrad Kangli Emery
Conrad, Kangli Emery
Kangli Emery Conrad
Conrad Kangli Emery Q
Mun Jern Wei
Mun, Jern Wei Ivan
Mun, Ivan
Wong Yijie Lucas
Wong, Yijie Lucas
Tung Hon Andrew Pak
Pak, Andrew
Brandon Heng Fu Hong
Heng, Brandon
Brandon Heng Fu Hong
HENG, BRANDON
HENG, FU HONG, BRANDON
HENG, FU HONG BRANDON
Mohamed, Haja Fayiz
Osman, Amir Rusyaidi
nan
Goh, Shaun
Shaun Goh
Ethan Yan
TAN, AARON JUSTIN
Tan, Aaron Justin
Rui Yong Soh
YEE, ERIC CHUN WAI
Yee, Eric
YEE CHUN WAI, ERIC
Eric Yee Chun Wai
Yee Chun Wai, Eric
Yee, Chun Wai, Eric
Yee Chun Wai, Eric
YEE, ERIC CHUN WAI
Lim, Oliver
Jie Cong Jayden, Tan
TAN , JI